In [1]:
cd /content/drive/MyDrive/코드정리(블로그)/DL/Test/SimCSE/

/content/drive/MyDrive/코드정리(블로그)/DL/Test/SimCSE


In [3]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 66.2 MB/s 
     |████████████████████████████████| 596 kB 67.9 MB/s 
     |████████████████████████████████| 6.8 MB 80.5 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
import torch.optim as optim
from torch import nn

from transformers import AutoTokenizer

from data.dataloader import ModelDataLoader, get_loader
from model.model import SimCSE
from model.loss import Loss
from model.utils import Metric
from trainer import Trainer, Tester
from SemanticSearch import semantic_search

In [3]:
args={}
args['model_name']='klue/bert-base'
args['weight_path']='weights/'
args['test_model_name']=''
args['test_tokenizer']=''
args['path_to_train_data']='data/train_nli.tsv'
args['path_to_valid_data']='data/valid_sts.tsv'
args['path_to_test_data']='data/test_sts.tsv'
args['device']="cuda" if torch.cuda.is_available() else "cpu"
args['temperature']=0.05
args['batch_size']=128
args['max_epochs']=3
args['learning_rate']=5e-5

In [ ]:
data_loader=get_loader(args,'train')

In [ ]:
model=SimCSE(args, mode='train').to(args['device'])

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
  {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
    'weight_decay': 0.01},
  {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
    'weight_decay': 0.0}
]

args['criterion']= nn.CrossEntropyLoss()
args['optimizer'] = optim.AdamW(optimizer_grouped_parameters, lr=args['learning_rate'])

loss=Loss(args)
metric=Metric(args)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
args

{'batch_size': 128,
 'criterion': CrossEntropyLoss(),
 'device': 'cuda',
 'learning_rate': 5e-05,
 'max_epochs': 1,
 'model_name': 'klue/bert-base',
 'optimizer': AdamW (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 5e-05
     weight_decay: 0.01
 
 Parameter Group 1
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 5e-05
     weight_decay: 0.0
 ),
 'path_to_test_data': 'data/test_sts.tsv',
 'path_to_train_data': 'data/train_nli.tsv',
 'path_to_valid_data': 'data/valid_sts.tsv',
 'temperature': 0.05,
 'test_model_name': 'klue_bert-base_epochs:1_cosim:0.9223',
 'test_tokenizer': 'klue/bert-base',
 'weight_path': 'weights/'}

In [ ]:
Trainer(args, data_loader, model, loss, metric)


======== Epoch 1 / 1 ========
Total Batch 3774/3774 , elapsed time : 0.9s , train_loss : 0.71
Validation Batch 12/12

Cosine-Similarity :	Pearson: 0.9211	Spearman: 0.9223
Manhattan-Distance:	Pearson: 0.9119	Spearman: 0.9119
Euclidean-Distance:	Pearson: 0.9080	Spearman: 0.9082
Dot-Product-Similarity:	Pearson: 0.9066	Spearman: 0.8979



In [ ]:
args['test_model_name']='klue_bert-base_epochs:1_cosim:0.9223'
args['test_tokenizer']='klue/bert-base'

Tester(args, loss, metric)

Test Batch 11/11

Cosine-Similarity :	Pearson: 0.8735	Spearman: 0.8625
Manhattan-Distance:	Pearson: 0.8793	Spearman: 0.8634
Euclidean-Distance:	Pearson: 0.8762	Spearman: 0.8618
Dot-Product-Similarity:	Pearson: 0.8386	Spearman: 0.8233



In [5]:
args['test_model_name']='klue_bert-base_epochs:1_cosim:0.9223'
args['test_tokenizer']='klue/bert-base'

model=SimCSE(args, mode='test').to(args['device'])
tokenizer=AutoTokenizer.from_pretrained(args['test_tokenizer'])

# Corpus with example sentences
corpus = ['한 남자가 음식을 먹는다.',
          '한 남자가 빵 한 조각을 먹는다.',
          '그 여자가 아이를 돌본다.',
          '한 남자가 말을 탄다.',
          '한 여자가 바이올린을 연주한다.',
          '두 남자가 수레를 숲 속으로 밀었다.',
          '한 남자가 담으로 싸인 땅에서 백마를 타고 있다.',
          '원숭이 한 마리가 드럼을 연주한다.',
          '치타 한 마리가 먹이 뒤에서 달리고 있다.']

# Corpus with example sentences
queries = ['한 남자가 파스타를 먹는다.',
           '고릴라 의상을 입은 누군가가 드럼을 연주하고 있다.',
           '치타가 들판을 가로 질러 먹이를 쫓는다.']

In [9]:
semantic_search(5, args, tokenizer, model, corpus, queries)



Query: 한 남자가 파스타를 먹는다.

Top 5 most similar sentences in corpus:

한 남자가 음식을 먹는다. (Score: 0.5947)
한 남자가 빵 한 조각을 먹는다. (Score: 0.5746)
원숭이 한 마리가 드럼을 연주한다. (Score: 0.1356)
치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.1138)
한 남자가 말을 탄다. (Score: 0.0623)


Query: 고릴라 의상을 입은 누군가가 드럼을 연주하고 있다.

Top 5 most similar sentences in corpus:

원숭이 한 마리가 드럼을 연주한다. (Score: 0.6212)
한 남자가 말을 탄다. (Score: 0.3265)
한 남자가 담으로 싸인 땅에서 백마를 타고 있다. (Score: 0.2784)
한 여자가 바이올린을 연주한다. (Score: 0.2616)
치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.2007)


Query: 치타가 들판을 가로 질러 먹이를 쫓는다.

Top 5 most similar sentences in corpus:

치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.6992)
한 남자가 음식을 먹는다. (Score: 0.2394)
원숭이 한 마리가 드럼을 연주한다. (Score: 0.1548)
한 남자가 말을 탄다. (Score: 0.0842)
한 남자가 빵 한 조각을 먹는다. (Score: 0.0593)
